In [16]:
import pandas as pd

In [17]:
DICT_MESES = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}


In [18]:
path_limpos = "Tratados"
path_brutos = "Brutos"

In [19]:
path_ena_bruto = path_brutos+"\\Simples_Energia_Natural_Afluente_Subsistema_Barra__data.csv"
path_carga_bruto = path_brutos+"\\Simples_Carga_de_Energia_Barra_Mês_data.csv"
path_vazao_bruto = path_brutos+"\\vazao_mensal_juazeiro_sf.csv"

In [20]:
ena_bruto = pd.read_csv(path_ena_bruto)
carga_bruto = pd.read_csv(path_carga_bruto)
vazao_bruto = pd.read_csv(path_vazao_bruto,on_bad_lines='skip',sep=";")

In [21]:
datas_brutas_ena = ena_bruto.iloc[:,0]
datas_brutas_carga= carga_bruto.iloc[:,0]

In [22]:
def limpa_datas_ena_carga(datas_brutas, DICT_MESES):

    df_limpo = pd.DataFrame()
    meses_brutos = datas_brutas.apply(lambda x: x.split(" ")[0])
    anos_brutos = datas_brutas.apply(lambda x: x.split(" ")[1])

    df_limpo["Month"] = meses_brutos.apply(lambda x: DICT_MESES[x])
    df_limpo["Year"] = anos_brutos

    df_limpo["Data"] = pd.to_datetime(df_limpo[["Month", "Year"]].assign(day=1))
    df_limpo.drop(["Month","Year"],axis=1,inplace=True)

    return df_limpo

In [23]:
df_ena_limpo = limpa_datas_ena_carga(datas_brutas_ena, DICT_MESES)
df_carga_limpo = limpa_datas_ena_carga(datas_brutas_carga, DICT_MESES)

In [24]:
df_ena_limpo["ENA"] = ena_bruto.iloc[:,-1]
df_carga_limpo["Carga"] = carga_bruto.iloc[:,-1]


In [26]:
df_carga_limpo = df_carga_limpo[df_carga_limpo.Data.dt.year >= 2002]
df_carga_limpo

,Data,Carga
0,2023-06-01,39917.917295
1,2023-05-01,41246.681730
2,2023-04-01,40279.185284
3,2023-03-01,43688.792105
4,2023-02-01,43029.632306
...,...,...
253,2002-05-01,24774.471236
254,2002-04-01,25772.850000
255,2002-03-01,25133.201223
256,2002-02-01,23452.702202


In [27]:
df_ena_limpo.sort_values(by="Data",inplace=True)
df_carga_limpo.sort_values(by="Data",inplace=True)

In [28]:
df_ena_limpo.to_csv(path_limpos+"\\ena_limpo.csv",index=False)
df_carga_limpo.to_csv(path_limpos+"\\carga_limpo.csv",index=False)

In [13]:
vazao_limpo = vazao_bruto[["Data","Media"]]
vazao_limpo.loc[:,"Data"] = pd.to_datetime(vazao_limpo.loc[:,"Data"])

mask_datas = vazao_limpo["Data"]>=pd.to_datetime("01/01/2000")
vazao_limpo = vazao_limpo[mask_datas]

vazao_limpo.columns=["Data","Vazao"]

C:\Users\matno\AppData\Local\Temp\ipykernel_8168\3197090982.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vazao_limpo.loc[:,"Data"] = pd.to_datetime(vazao_limpo.loc[:,"Data"])


In [14]:
vazao_limpo.sort_values(by="Data",inplace=True)
vazao_limpo.to_csv(path_limpos+"\\vazao_limpo.csv",index=False)